In [1]:
import pandas as pd
import numpy as np

In [2]:
arr = pd.array([1, 2, None], dtype=pd.Int64Dtype())


In [3]:
# CSV data from https://www.data.gouv.fr/fr/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2019/

# PDF data description here for 2019 : https://www.data.gouv.fr/fr/datasets/r/6cade01c-f69d-4779-b0a4-20606069888f
# PDF data description here for 2005-2108 : https://www.data.gouv.fr/fr/datasets/r/8d4df329-bbbb-434c-9f1f-596d78ad529f

csv_format_2019 = {

    'separator' : ';',
    'decimal' : ',',
    'encoding' : 'utf-8',
    'scale_lat_log' : 1,
    'dtype' : {
        'Num_Acc' : 'Int64',
    },
    'na_values': None,
}

csv_format_2015_2018 = {
    'separator' : ',',
    'decimal' : '.',
    'encoding' : 'latin_1',
    'scale_lat_log' : 1/100000,
    'dtype' : {
        'voie' :'str',
    },
    'na_values': None,
}

data_sources = [ 
    {
        'année': 2019,
        'caracteristiques': 'https://www.data.gouv.fr/fr/datasets/r/e22ba475-45a3-46ac-a0f7-9ca9ed1e283a',
        'lieux' : 'https://www.data.gouv.fr/fr/datasets/r/2ad65965-36a1-4452-9c08-61a6c874e3e6',
        'véhicules': 'https://www.data.gouv.fr/fr/datasets/r/780cd335-5048-4bd6-a841-105b44eb2667',
        'usagers' :'https://www.data.gouv.fr/fr/datasets/r/36b1b7b3-84b4-4901-9163-59ae8a9e3028',
    },
    {
        'année': 2018,
        'caracteristiques': 'https://www.data.gouv.fr/fr/datasets/r/6eee0852-cbd7-447e-bd70-37c433029405',
        'lieux' : 'https://www.data.gouv.fr/fr/datasets/r/d9d65ca1-16a3-4ea3-b7c8-2412c92b69d9',
        'véhicules': 'https://www.data.gouv.fr/fr/datasets/r/b4aaeede-1a80-4d76-8f97-543dad479167',
        'usagers' : 'https://www.data.gouv.fr/fr/datasets/r/72b251e1-d5e1-4c46-a1c2-c65f1b26549a',
    },
    {
        'année': 2017,
        'caracteristiques': 'https://www.data.gouv.fr/fr/datasets/r/9a7d408b-dd72-4959-ae7d-c854ec505354',
        'lieux' : 'https://www.data.gouv.fr/fr/datasets/r/9b76a7b6-3eef-4864-b2da-1834417e305c',
        'véhicules': 'https://www.data.gouv.fr/fr/datasets/r/d6103d0c-6db5-466f-b724-91cbea521533',
        'usagers' : 'https://www.data.gouv.fr/fr/datasets/r/07bfe612-0ad9-48ef-92d3-f5466f8465fe',
    },
    {
        'année': 2016,
        'caracteristiques': 'https://www.data.gouv.fr/fr/datasets/r/96aadc9f-0b55-4e9a-a70e-c627ed97e6f7',
        'lieux' : 'https://www.data.gouv.fr/fr/datasets/r/08b77510-39c4-4761-bf02-19457264790f',
        'véhicules': 'https://www.data.gouv.fr/fr/datasets/r/be2191a6-a7cd-446f-a9fc-8d698688eb9e',
        'usagers' : 'https://www.data.gouv.fr/fr/datasets/r/e4c6f4fe-7c68-4a1d-9bb6-b0f1f5d45526',
    },
    {
        'année': 2015,
        'caracteristiques': 'https://www.data.gouv.fr/fr/datasets/r/185fbdc7-d4c5-4522-888e-ac9550718f71',
        'lieux' : 'https://www.data.gouv.fr/fr/datasets/r/31db21ef-4328-4c5e-bf3d-66a8fe82e6a2',
        'véhicules': 'https://www.data.gouv.fr/fr/datasets/r/3420157e-7d23-4832-a710-a3a2f2df909c',
        'usagers' : 'https://www.data.gouv.fr/fr/datasets/r/3420157e-7d23-4832-a710-a3a2f2df909c',
    }
]

In [4]:
from os.path import exists

dfs={}
for data in data_sources:
    
    year = data['année']
    print(f"download year={year}")
    if year < 2019:
        csv_format = csv_format_2015_2018
    else:
        csv_format = csv_format_2019
    dfs_y = {}
    for key in data.keys():
        if key == 'année':
            continue
        print(f" data base '{key}'")
        filename = f'df_{year}_{key}.ftr'
        if exists(filename):
            print(f'load local {filename}')
            dfs_y[key] = pd.read_feather(filename)
        else:
            print(f'download data for {year} {key}')
            dfs_y[key]= pd.read_csv(
                data[key],
                sep=csv_format['separator'],
                decimal=csv_format['decimal'],
                encoding=csv_format['encoding'],
                dtype=csv_format['dtype'],
                na_values=csv_format['na_values']
            )
            #dfs_y[key].replace('NaN',np.NaN)
            # scale Latitude and logitude
            for l in ['lat', 'long']:
                if l in dfs_y[key].keys():
                    print(f'scale {l}')
                    dfs_y[key][l] = dfs_y[key][l] * csv_format['scale_lat_log']
            # save dataframe in feather format because :
            # https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d
            print(f'save local {filename}')
            dfs_y[key].to_feather(filename)
        print(dfs_y[key].dtypes)
    dfs[year] = dfs_y

download year=2019
 data base 'caracteristiques'
load local df_2019_caracteristiques.ftr
Num_Acc      Int64
jour         int64
mois         int64
an           int64
hrmn        object
lum          int64
dep         object
com         object
agg          int64
int          int64
atm          int64
col          int64
adr         object
lat        float64
long       float64
dtype: object
 data base 'lieux'
load local df_2019_lieux.ftr
Num_Acc      Int64
catr         int64
voie        object
v1         float64
v2          object
circ         int64
nbv          int64
vosp         int64
prof         int64
pr          object
pr1         object
plan         int64
lartpc      object
larrout     object
surf         int64
infra        int64
situ         int64
vma          int64
dtype: object
 data base 'véhicules'
load local df_2019_véhicules.ftr
Num_Acc          Int64
id_vehicule     object
num_veh         object
senc             int64
catv             int64
obs              int64
obsm          

In [5]:
dfs[2019]['caracteristiques']

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,201900000001,30,11,2019,01:30,4,93,93053,1,1,1,2,AUTOROUTE A3,48.896210,2.470120
1,201900000002,30,11,2019,02:50,3,93,93066,1,1,1,6,AUTOROUTE A1,48.930700,2.368800
2,201900000003,28,11,2019,15:15,1,92,92036,1,1,1,4,AUTOROUTE A86,48.935872,2.319174
3,201900000004,30,11,2019,20:20,5,94,94069,1,1,1,4,A4,48.817329,2.428150
4,201900000005,30,11,2019,04:00,3,94,94028,1,1,1,2,A86 INT,48.776362,2.433254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58835,201900058836,30,11,2019,09:00,1,69,69288,1,1,5,2,A43 13.029 A 15.971,45.666660,5.056120
58836,201900058837,27,11,2019,07:50,1,67,67482,1,1,8,4,Autoroute A.35,48.576900,7.726900
58837,201900058838,30,11,2019,02:41,4,94,94021,1,1,1,6,AUTOROUTE A6A,48.771700,2.345760
58838,201900058839,30,11,2019,15:20,1,78,78640,1,1,1,7,A86 EXT,48.777289,2.223759


In [8]:
dfs[2018]['caracteristiques']

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep
0,201800000001,18,1,24,1505,1,1,4,1.0,1.0,5,route des Ansereuilles,M,50.55737,2.94992,590
1,201800000002,18,2,12,1015,1,2,7,7.0,7.0,11,Place du général de Gaul,M,50.52936,2.93151,590
2,201800000003,18,3,4,1135,1,2,3,1.0,7.0,477,Rue nationale,M,50.51243,2.91714,590
3,201800000004,18,5,5,1735,1,2,1,7.0,3.0,52,30 rue Jules Guesde,M,50.51974,2.89123,590
4,201800000005,18,6,26,1605,1,2,1,1.0,3.0,477,72 rue Victor Hugo,M,50.51607,2.90605,590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57778,201800057779,18,12,15,1050,1,2,1,1.0,6.0,611,passamanty-Sada ( rue de,Y,12.80217,45.21356,976
57779,201800057780,18,12,16,610,2,2,1,1.0,3.0,611,mamoudzou centre-Command,Y,12.78120,45.23436,976
57780,201800057781,18,12,22,2150,5,2,1,1.0,6.0,611,kaweni-SPPM ( rue de la,Y,12.76930,45.22409,976
57781,201800057782,18,12,23,1945,5,2,1,1.0,1.0,611,cavani-Kavani sud ( rue,Y,12.78622,45.22036,976
